[Markdown Guide](https://docs.gitlab.com/ee/user/markdown.html)

# Praxismodul I
## Thema 6: Process Organisation
## Hochschule Mainz
## Wirtschaftsinformatik dual B.Sc., Fachsemester 4

Diese Arbeit wurde von folgenden Studenten durchgeführt:

| Student | Matrikel-Nummer |
|:-- |:--|
|Fabian Harmsen| 943611|
|Till Waller   | 943565|
|Moritz Wetzel | 943566|
Tabelle 1: Studenten der durchgeführten Arbeit

## Gliederung

1. Projektziel
2. Soziale Netzwerke
3. Erklärungen zu den grafischen Ergebnissen
4. Configuration
5. Import
6. Similar Activities
7. Discover Roles
8. Handover Of Work
9. Clustering Similar Activities
10. Clustering Working together
11. Fazit

## 1. Projektziel

Im Rahmen dieser Projektarbeit wird die Prozessorganisation eines zur Verfügung gestellten Datensatzes untersucht. Die uns zur Verfügung gestellte Logdatei stammt aus der BPI-Challenge 2017 und bestehen aus 2 Geschäftsprozessen. Unser Projektziel wird daraus bestehen, soziale Netzwerke innerhalb eines gewünschten Datensatzes zu entdecken. Hierfür werden die Daten mit Methoden der sozialen Netzwerkanalyse untersucht, um die sozialen Verbindungen im importierten Datensatz zu untersuchen und auszuwerten. Die Ergebnisse werden mithilfe eines Jupyter-Notebooks berechnet und visuell dargestellt. Zum Ende des Projekts werden zwei weitere Datensätze mit den implementierten Analysen untersucht, um die Funktionalität der Methodik zu validieren.

## 2 Soziale Netzwerke

Der Begriff `Soziales Netzwerk` bestimmt sich durch die Interaktion und Kommunikation zwischen mehreren realen Personen in einem privaten oder öffentlichen Umfeld. Im Rahmen unseres Projektes werden dabei ausschließliche `Soziale Netzwerke` im Unternehmensumfeld betrachtet.

Bei der Analyse der Personengruppe zeichnen sich Schlüsselpersonen ab, die die verschiedenen Personengruppen im `Sozialen Netzwerk` zusammenhalten. Diese Analysen sind für Unternehmen besonders hilfreich, da sich neben der Unternehmensorganisation im Organigramm die reale Struktur eines Unternehmens erkennen lässt.

## 3. Erklärung zu den grafischen Ergebnissen

Dieser Abschnitt erklärt die grafischen Ergebnisse unabhängig von angewandten Methoden beziehungsweise Analysen. Die Ausgaben setzen sich aus Nodes (Knoten) und Edges (Verbindungen) zusammen. Ein Node stellt im Rahmen unserer Arbeit einen Nutzer dar. Nach dem Clustering stellt ein Node eine Benutzergruppe dar. Die Benutzer beziehungsweise Benutzergruppen sind unterschiedlich stark über Edges miteinander verbunden. Die Stärke dieser Verbindung wird über Analysen festgestellt.

Aus Nodes und Edges werden Physiksimulationen erstellt, welche den visuellen Darstellungen der Methodenergebnisse dienen. Die Edges wirken wie Gummibänder, die die Nodes zusammenhalten. Die Nodes hingegen versuchen möglichst viel Abstand zu anderen Nodes zu gewinnen. Dies führt dazu, dass Nodes, die aufgrund der Analysen mit starken Edges verbunden sind, sich annähern und weniger stark verbundene Nodes sich voneinander entfernen. Sobald sich die Physiksimulation beruhigt hat, kann man Gruppen von Nodes erkennen sowie die Verbindungen zu anderen Gruppen.

Wir bieten zwei Methoden zur Erstellung von grafischen Ausgaben:
 - `NetworkX` wird direkt im Notebook ausgegeben,
 - `Pyvis` erzeugt genauere webbasierte Ausgaben.

Die `Pyvis`-Analyseergebnisse können als HTML-Datei im Browser geöffnet werden und sind unter "result/*" zu finden.

*Das Programm erzeugt während beziehungsweise nach dem Clustering keine grafische Ausgabe, da dies bei den getesteten Datensätzen keine neuen Erkenntnisse gebracht hat.*

## 4. Configuration

Die folgende Tabelle erklärt die Variablen, die im nächsten Code-Block definiert werden können. Die Logdatei muss, um richtig analysiert werden zu können, im XES-Dateiformat vorliegen. Die Datei kann über den FileDialog des Betriebssystems ausgewählt werden.

| variable | type | description| default|
|:--|:--|:--|:--|
|log_path | (path)| path to the XES File|`data/test.xes`|
|use_networkx | (bool)| output via NetworkX or Pyvis| `False`|
|filter_timeframe | (bool)| filter the timeframe of the traces| `False` |
|filter_timeframe_intersecting | (bool)| use intersecting or contained| `False` |
|filter_timeframe_start | (dateTime)| start of the Timeframe| `1970-01-01 00:00:00` |
|filter_timeframe_end | (dateTime)| end of the Timeframe| `2038-12-31 00:00:00` |
|filter_performance| (bool)| filter on Traceperformance| `False` |
|filter_performance_min| (int)| minimum seconds of trace actions|`0` |
|filter_performance_max |(int)| maximum seconds of trace actions|`360000` |
Tabelle 2: Benötigte Konfigurationsvariablen für den Import

Die Option `Pyvis` legt die Ergebnisse im Ordner `./result` ab, welcher schreibend Zugriff benötigt.

In [ ]:
from tkinter import filedialog

#log_path = 'data/BPI_Challenge_2019.xes'
log_path = filedialog.askopenfilename(initialdir="/",
                                      title="Select a File",
                                      filetypes=(
                                          ("eXtensible Event Stream", "*.xes*"),
                                          ("all files", "*.*"))
                                      )

use_networkx = False

filter_timeframe = False
filter_timeframe_intersecting = False
filter_timeframe_start = "2011-03-09 00:00:00"
filter_timeframe_end = "2011-03-09 00:00:00"

filter_performance = False
filter_performance_min = 0
filter_performance_max = 864000

## 5. Import
Im folgenden Codeblock werden alle Bibliotheken geladen die im laufenden Skript benötigt werden. Wenn die Bibliotheken nicht gefunden werden können, können sie über den Packagemanager pip seperat nachinstalliert werden.

**Liste der benötigten Pythonpackete (pip)**
 - pm4py
 - shutil
 - numpy
 - sklearn
 - scipy
 - tkinter


In [ ]:
import shutil
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.organizational_mining.sna import algorithm as sna
from pm4py.visualization.sna import visualizer as sna_visualizer
from pm4py.algo.organizational_mining.roles import algorithm as roles_discovery
from pm4py.algo.organizational_mining.sna import util
from pm4py.algo.filtering.log.timestamp import timestamp_filter
from pm4py.algo.filtering.log.cases import case_filter
from pathlib import Path
#from tkinter import filedialog (see above)

Der Ordner für die webbasierten Exportergebnisse wird in diesem Codeblock erstellt. Wird NetworkX als Ausgabe genutzt, wird dieser Abschnitt übersprungen.


In [ ]:
if not use_networkx:
    Path("./result").mkdir(parents=True, exist_ok=True)

Dieser Code-Block lädt den Datensatz und wendet die eingestellten Filter an.
Die Einstellungen dazu wurden in Block 3 (Configuration) festgelegt.

In [ ]:
log = xes_importer.apply(log_path)

if filter_timeframe:
    if not filter_timeframe_intersecting:
        log = timestamp_filter.filter_traces_contained(log, filter_timeframe_start, filter_timeframe_end)
    else:
        log = timestamp_filter.filter_traces_intersecting(log, filter_timeframe_start, filter_timeframe_end)

if filter_performance:
    log = case_filter.filter_case_performance(log, filter_performance_min, filter_performance_max)

## 6. Similar Activities

In diesem Abschnitt wird die Methode `Similar Activities` auf den zuvor importierten Datensatz angewendet, um zu berechnen, wie sehr sich die Arbeitsabläufe zwischen den Nutzern (Nodes) ähneln (Quelle: https://pm4py.fit.fraunhofer.de/documentation).

In [ ]:
sa_metric = sna.apply(log, variant=sna.Variants.JOINTACTIVITIES_LOG)

if use_networkx:
    gviz_ja_py = sna_visualizer.apply(sa_metric, variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_ja_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_ja_py = sna_visualizer.apply(sa_metric, variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_ja_py, "result/similar_activities.html")

## 7. Discover Roles

In diesem Abschnitt wird die Methode `Discover Roles` auf den zuvor importierten Datensatz angewendet, um die Rollen eines Nutzers (Node) anhand seiner durchgeführten Aktivitäten zu erkennen. Eine Rolle ist dabei ein Satz von Arbeitsaktivitäten im Log, die in ähnlicher Weise von einem Nutzer (Node) oder mehreren Nutzern (Nodes) ausgeführt wird. Zu Beginn der Analyse gehört jede Arbeitsaktivität zu unterschiedlichen Positionen und ist mit einer hohen Anzahl an Urhebern verbunden. Anschließend werden die Rollen entsprechend ihrer Ähnlichkeit der Arbeitsabläufe gruppiert, bis keine weitere Gruppierung mehr möglich ist. (Quelle: https://pm4py.fit.fraunhofer.de/documentation)

Folgende Quelle kann helfen, die aufgeführten Rollen zu verstehen und zu erkennen:

Burattin, Andrea, Alessandro Sperduti, and Marco Veluscek. “Business models enhancement through discovery of roles.” 2013 IEEE Symposium on Computational Intelligence and Data Mining (CIDM). IEEE, 2013

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Rollenname auf Basis der Aktivitäten
2. Benutzer mit der Angabe wie aktiv der Nutzer in der Rolle war
3. `Leerzeile`

In [ ]:
roles = roles_discovery.apply(log)

for x in roles:
    print(x[0])
    print(x[1])
    print("")

## 8. Handover Of Work

In diesem Abschnitt wird die Methode `Handover Of Work` auf den zuvor importierten Datensatz angewendet, um zu messen, wie oft ein Nutzer (Node) bei der Ausführung eines Geschäftsprozesses einer anderen Person die Arbeit übergibt.
Bei einem Handover übergibt ein Nutzer den Geschäftsvorfall am Ende seiner Aktivität einem anderen Nutzer. Die Stärke der Beziehungen (Edges) ist abhängig von der Anzahl der durchgeführten Handovers. Die Edges zeigen bei `Handover Of Work`-Analysen in die Richtung des Arbeitsflusses.

 (Quelle: https://pm4py.fit.fraunhofer.de/documentation)



In [ ]:
hw_values = sna.apply(log, variant=sna.Variants.HANDOVER_LOG)

if use_networkx:
    gviz_hw_py = sna_visualizer.apply(hw_values, variant=sna_visualizer.Variants.NETWORKX)
    sna_visualizer.view(gviz_hw_py, variant=sna_visualizer.Variants.NETWORKX)
else:
    gviz_hw_py = sna_visualizer.apply(hw_values, variant=sna_visualizer.Variants.PYVIS)
    shutil.move(gviz_hw_py, "result/handover_of_work.html")

## 9. Clustering Similar Activities

In diesem Abschnitt werden die unter 6 analysierten Nutzer in Nutzergruppen zusammengefasst, welche gleiche Aufgaben verrichtet haben. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [ ]:
clustering_sa = util.cluster_affinity_propagation(sa_metric)

for group in clustering_sa:
    print("Gruppe: " + group)
    print(clustering_sa[group])
    print("")


## 10. Clustering Handover Of Work

In diesem Abschnitt werden die unter 8 analysierten Nutzer als Gruppe zusammengefasst, welche häufig miteinander arbeiten. Dies soll die Übersichtlichkeit erhöhen und erste Schlussfolgerungen ermöglichen.

Die Analyseergebnisse werden folgendermaßen ausgegeben:

1. Cluster
2. Benutzerliste
3. `Leerzeile`

In [ ]:
clustering_hw = util.cluster_affinity_propagation(hw_values)

for group in clustering_hw:
    print("Gruppe: " + group)
    print(clustering_hw[group])
    print("")

## 11. Fazit

Im Projekt wurden verschiedene Datensätze im `eXtensible Event Stream`-Format ausgewertet. Dabei konnten mehrere Methoden der `Social Network Analysis` (SNA) erfolgreich durchgeführt werden.

Anhand der durchgeführten Analysen konnten einerseits Personengruppen erkannt werden, die regelmäßig ähnlich oder gleiche Aktivitäten vollzogen haben. Anderseits konnten Personen ermittelt werden, die wenig bis gar nicht in die `Sozialen Netzwerke` des Unternehmens eingebunden sind.
Auffällig war aber, dass es auch Geschäftsprozesse gab, bei denen alle Beteiligten ein großes Team bildeten. In solchen Fällen erzeugen die durchgeführten Analysen kein interpretierbares Ergebnis.


In [ ]:
print("Das Notebook wurde erfolgreich ausgeführt.")
